# Homework Assignment: Implement Word2Vec

In this assignment, you are required to implement the `Word2Vec` class from scratch using Python.

Please complete the methods inside the `Word2Vec` class:
- `__init__`: Initialize weights
- `softmax`: Impliment softmax activation function
- `train`: Loop over the data and optimize your model

You may use NumPy, but **do not** use any external word embedding libraries like Gensim or Torch for this task.

Good luck!


# Skip Gram Implementation

In [13]:
import re
import numpy as np
from collections import defaultdict
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [2]:
class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.word_count = defaultdict(int)
        self.total_words = 0
        self.vocab_size = 0

    def build_vocab(self, sentences, min_count=2):
        # Count words
        for word in sentences:
            self.word_count[word] += 1

        # Create word2idx and idx2word mapping
        idx = 0
        for word, count in self.word_count.items():
            if count >= min_count:
              self.word2idx.update({word: idx})
              idx += 1

        # self.word2idx = {word: idx for idx, (word, count) in enumerate(self.word_count.items()) if count >= min_count}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
        self.total_words = sum([count for word, count in self.word_count.items() if count >= min_count])

    def word_to_index(self, word):
        return self.word2idx.get(word, -1)

    def index_to_word(self, index):
        return self.idx2word.get(index, None)


In [3]:
def generate_training_data(vocab, sentences, window_size=2):
    training_data = []
    sentence_indices = [vocab.word_to_index(word) for word in sentences if vocab.word_to_index(word) != -1]

    for center_idx, center_word in enumerate(sentence_indices):
        context_start = max(0, center_idx - window_size)
        context_end = min(len(sentence_indices), center_idx + window_size + 1)

        for context_idx in range(context_start, context_end):
            if context_idx != center_idx:
                context_word = sentence_indices[context_idx]
                training_data.append((center_word, context_word))

    return np.array(training_data)


In [ ]:
class Word2Vec:
    def __init__(self, vocab_size, embed_size=100, learning_rate=0.001):
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.learning_rate = learning_rate

        # Initialize weight matrices
        self.W = np.random.uniform(-0.5, 0.5, (vocab_size, embed_size))  # Input to hidden
        self.W_prime = np.random.uniform(-0.5, 0.5, (embed_size, vocab_size))  # Hidden to output

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x)

    def train(self, training_data, epochs=1000):
        for epoch in range(epochs):
            loss = 0
            for center_word, context_word in training_data:
                h = self.W[center_word]  # Get hidden layer (input->hidden)
                u = np.dot(h, self.W_prime)  # Predict output
                y_pred = self.softmax(u)

                # One-hot encoding the true context word
                y_true = np.zeros(self.vocab_size)
                y_true[context_word] = 1

                # Calculate the error
                error = y_pred - y_true

                # Update weights with gradient descent
                self.W_prime -= self.learning_rate * np.outer(h, error)
                self.W[center_word] -= self.learning_rate * np.dot(self.W_prime, error)

                # Calculate loss (cross-entropy)
                loss -= np.log(y_pred[context_word])

            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')




In [ ]:
# Reading the Persian stopwords from the file
with open('persian.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())  # Using a set for faster lookup

# Sample Persian text
text = "ملکه و زن ها در کنار همسران و خانواده خود یعنی شاه و مرد ها در یک سرزمین پهناور زندگی می‌کردند شاه همیشه به مرد ها تذکر میداد که قدرت در اتحاد مرد ها و شاه نهفته است و در این قلمرو ملکه به زن ها یادآوری می‌کرد که همبستگی زن ها و ملکه مهم است و در این داستان هر مرد که نزد شاه یا زن که نزد ملکه می‌آمد از آنها حکم می‌گرفت تا به دیگران کمک کنند شاه عادل و قادر بود و ملکه خردمند و زیبا و هر مرد که از حکمت شاه یا زن که از عدالت ملکه راضی نبود نزد آنها می‌رفت تا شکایت خود را مطرح کند شاه و مرد و ملکه و زن در کنار هم بودند و هیچ کس از شاه یا ملکه نمی‌ترسید شاه همیشه به زبان میاورد که مرد ها باید به یکدیگر کمک کنند و ملکه تأکید داشتند زن ها هم باید متحد باشند"

# Tokenizing the text (you can modify the tokenizer if needed)
words = text.split()

# Removing stopwords
filtered_text = [word for word in words if word not in stopwords]

# Joining the words back into a sentence
cleaned_text = ' '.join(filtered_text)

print(cleaned_text)


ملکه زن همسران خانواده شاه مرد سرزمین پهناور زندگی می‌کردند شاه مرد تذکر میداد قدرت اتحاد مرد شاه نهفته قلمرو ملکه زن یادآوری می‌کرد همبستگی زن ملکه مهم داستان مرد شاه زن ملکه می‌آمد حکم می‌گرفت کمک شاه عادل قادر ملکه خردمند زیبا مرد حکمت شاه زن عدالت ملکه راضی می‌رفت شکایت مطرح شاه مرد ملکه زن شاه ملکه نمی‌ترسید شاه زبان میاورد مرد کمک ملکه تأکید زن متحد


In [ ]:
# wiki_dump_path = 'enwiki-latest-pages-articles.xml.bz2'  # Path to your Wikipedia dump file

# # Load and preprocess the dataset
# sentences = list(load_wiki_data(wiki_dump_path))

# Build the vocabulary
vocab = Vocabulary()
vocab.build_vocab(cleaned_text.split(' '))

# Generate training data
training_data = generate_training_data(vocab, cleaned_text.split(' '))

# Initialize and train Word2Vec model
word2vec_model = Word2Vec(vocab.vocab_size)
word2vec_model.train(training_data, epochs=1000)


Epoch 0, Loss: 217.34630970038413
Epoch 100, Loss: 181.5196690584462
Epoch 200, Loss: 181.10669945586082
Epoch 300, Loss: 180.9694558003934
Epoch 400, Loss: 180.89839648378222
Epoch 500, Loss: 180.85521332091994
Epoch 600, Loss: 180.82638982519742
Epoch 700, Loss: 180.80589790542393
Epoch 800, Loss: 180.7906516880482
Epoch 900, Loss: 180.77891218264517


Defines a function to retrieve the word embedding for a given word from a Word2Vec model. If the word exists in the vocabulary, its corresponding vector is returned; otherwise, None is returned.

In [ ]:
def get_word_embedding(word, vocab, model):
    word_idx = vocab.word_to_index(word)
    if word_idx != -1:
        return model.W[word_idx]
    else:
        return None

embedding = get_word_embedding("مرد", vocab, word2vec_model)
print(embedding)


[ 0.28634888 -0.13093086 -0.0201545   0.05271528 -0.09334632  0.15840051
 -0.34445462  0.42237267 -0.38221831 -0.19108261  0.47323099  0.4272571
  0.13851498 -0.22135522  0.23678668 -0.28319392  0.30872885  0.39926364
 -0.52409306 -0.22185257 -0.3983401  -0.16248012  0.2544923  -0.30862513
 -0.14724238  0.15341193  0.22308444 -0.03885356  0.50674215  0.3120602
 -0.22368227 -0.42046334  0.34820725 -0.11986774 -0.39504836 -0.23070816
 -0.40190402  0.41267271 -0.07634545 -0.19427366  0.22134564 -0.38971622
 -0.54359274  0.44488732  0.1493101  -0.42904048  0.24654613 -0.26918437
  0.02520455 -0.3487133   0.34302321 -0.32930649 -0.26991703  0.21795934
 -0.30534359 -0.08726335  0.40263831  0.10635999 -0.3494115  -0.16635264
  0.40776723 -0.01813031  0.21182939 -0.40908686  0.2195951   0.39119267
 -0.2490812   0.3413859   0.20336299 -0.11541427  0.00648979 -0.27482335
 -0.37028258  0.2318869   0.46906915 -0.27019587 -0.18769502 -0.00147952
 -0.31550756  0.12446    -0.17474903 -0.36052425  0.0